In [0]:
! wget -O init_env.sh https://www.dropbox.com/s/6bnwn8u2hz19s59/init_env.sh && \
bash init_env.sh

--2019-11-12 02:48:21--  https://www.dropbox.com/s/6bnwn8u2hz19s59/init_env.sh
Resolving www.dropbox.com (www.dropbox.com)... 162.125.9.1, 2620:100:601f:1::a27d:901
Connecting to www.dropbox.com (www.dropbox.com)|162.125.9.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/6bnwn8u2hz19s59/init_env.sh [following]
--2019-11-12 02:48:21--  https://www.dropbox.com/s/raw/6bnwn8u2hz19s59/init_env.sh
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uce6759efc406346f17db56a7663.dl.dropboxusercontent.com/cd/0/inline/AsMAvqzTcuSfbDaMfSVucJuoe51Y0GuYR9LHyJdMh98DF6NJGsFo532WN4w4l_BtqZRs15Gm6_rowvwHiX-N7QX_QLrMILeE0sgnTnoDl4MGDQ/file# [following]
--2019-11-12 02:48:21--  https://uce6759efc406346f17db56a7663.dl.dropboxusercontent.com/cd/0/inline/AsMAvqzTcuSfbDaMfSVucJuoe51Y0GuYR9LHyJdMh98DF6NJGsFo532WN4w4l_BtqZRs15Gm6_rowvwHiX-N7QX_QLrMILeE0sgnTnoDl4MGDQ/file
Resolving uce6759efc40

In [0]:
import os, sys
os.environ['SPARK_HOME'] = "/usr/local/spark"
os.environ['PYSPARK_PYTHON'] = "/usr/local/bin/python"
sys.path.append("/usr/local/spark/python/")
sys.path.append("/usr/local/spark/python/lib/pyspark.zip")
sys.path.append("/usr/local/spark/python/lib/py4j-0.10.4-src.zip")

In [0]:
from pyspark import SparkContext
from pyspark import SparkConf
sc =SparkContext()

In [0]:
!wget -O pm25.csv "https://www.dropbox.com/s/zkn3ba7pitv83el/pm2.5Taiwan.csv?dl=0"

--2019-11-12 02:48:27--  https://www.dropbox.com/s/zkn3ba7pitv83el/pm2.5Taiwan.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.9.1, 2620:100:601f:1::a27d:901
Connecting to www.dropbox.com (www.dropbox.com)|162.125.9.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/zkn3ba7pitv83el/pm2.5Taiwan.csv [following]
--2019-11-12 02:48:27--  https://www.dropbox.com/s/raw/zkn3ba7pitv83el/pm2.5Taiwan.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc40c0f7302fbb766109152d7b23.dl.dropboxusercontent.com/cd/0/inline/AsObNdvh-TqkAMDBIiBukyXhuj9KFHhJvZln43weR6oTQ3CFMji-Y_czyIdrHNwQKZAjrQvauoeh3jzPRSCC9Qih-zBnUHhWL-6RlJ-E8DdrjA/file# [following]
--2019-11-12 02:48:28--  https://uc40c0f7302fbb766109152d7b23.dl.dropboxusercontent.com/cd/0/inline/AsObNdvh-TqkAMDBIiBukyXhuj9KFHhJvZln43weR6oTQ3CFMji-Y_czyIdrHNwQKZAjrQvauoeh3jzPRSCC9Qih-zBnUHhWL-6RlJ-E8DdrjA/file
Resol

In [0]:
weather = sc.textFile("./pm25.csv")

In [0]:
weather_data_rdd = weather.map(lambda line : line.split(","))
pm25schema = weather_data_rdd.first()
# print(pm25schema)

# 回想如何使用RDD計算求取2015年，大里每小時的平均pm25數值。

In [0]:
import math
def remove_row_with_noise (x):
    for i in range(3, len(x)):
        if not x[i].isdecimal():
            return False
    return True 

def hourKeyGen(x):
    hourkeypair = []
    x=x[3:]
    for i, value in enumerate(x):
      print(i, value)
      hourkeypair.append((i, float(value)))
    return hourkeypair

clean_weather_data = weather_data_rdd\
                    .filter(lambda x: x!=pm25schema)\
                    .filter(remove_row_with_noise)

In [0]:
dalipm25 = clean_weather_data.filter(lambda x: x[2]== "PM2.5")

In [0]:
count = dalipm25.count()
HourSum = dalipm25\
            .flatMap(hourKeyGen)\
            .reduceByKey(lambda x,y: x+y)\
            .mapValues(lambda x: x/count)\
            .map(lambda x: (x[1],x[0])).top(24)

print(HourSum)

[(24.46175554690541, 10), (24.37315103152978, 11), (24.045299727520437, 9), (23.94949396652394, 12), (23.225282210977035, 13), (23.090064227325808, 8), (22.805809653561695, 20), (22.70732775398988, 21), (22.61857726741923, 19), (22.479028804982484, 14), (22.383952899961074, 22), (22.223968470221877, 18), (21.990803814713896, 15), (21.943022576878164, 23), (21.835879719735306, 17), (21.78376800311405, 7), (21.783476060724016, 16), (21.535227715064227, 0), (21.192244063838068, 1), (20.72007590502141, 2), (20.610694822888284, 6), (20.346876216426626, 3), (20.115025301673803, 5), (20.07342351109381, 4)]


# 使用DataFrame 來計算每小時平均值

In [0]:
print(dalipm25.first())

['2015/01/01', '龍潭', 'PM2.5', '46', '71', '76', '74', '65', '62', '56', '50', '52', '56', '54', '47', '40', '36', '37', '27', '30', '25', '26', '24', '18', '16', '11', '14']


In [0]:
from pyspark.sql import SQLContext
from pyspark.sql import Row
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .getOrCreate()

In [0]:
from pyspark.sql import SQLContext
from pyspark.sql import Row

dalipm25row = dalipm25.map(lambda p:
        Row(
        date = p[0],
        location = p[1],
        measure = p[2],
        hr_01 = float(p[3]), hr_02 = float(p[4]),hr_03 = float(p[5]),hr_04 = float(p[6]),hr_05 = float(p[7]),
        hr_06 = float(p[8]), hr_07 = float(p[9]),hr_08 = float(p[10]),hr_09 = float(p[11]),hr_10 = float(p[12]),
        hr_11 = float(p[13]),hr_12 = float(p[14]),hr_13 = float(p[15]),hr_14 = float(p[16]),hr_15 = float(p[17]),
        hr_16 = float(p[18]),hr_17 = float(p[19]),hr_18 = float(p[20]),hr_19 = float(p[21]),hr_20 = float(p[22]),
        hr_21 = float(p[23]),hr_22 = float(p[24]),hr_23 = float(p[25]),hr_24 = float(p[26]),
    )
)

dalipm25row.take(5)

[Row(date='2015/01/01', hr_01=46.0, hr_02=71.0, hr_03=76.0, hr_04=74.0, hr_05=65.0, hr_06=62.0, hr_07=56.0, hr_08=50.0, hr_09=52.0, hr_10=56.0, hr_11=54.0, hr_12=47.0, hr_13=40.0, hr_14=36.0, hr_15=37.0, hr_16=27.0, hr_17=30.0, hr_18=25.0, hr_19=26.0, hr_20=24.0, hr_21=18.0, hr_22=16.0, hr_23=11.0, hr_24=14.0, location='龍潭', measure='PM2.5'),
 Row(date='2015/01/02', hr_01=15.0, hr_02=12.0, hr_03=9.0, hr_04=14.0, hr_05=17.0, hr_06=20.0, hr_07=18.0, hr_08=22.0, hr_09=21.0, hr_10=23.0, hr_11=18.0, hr_12=25.0, hr_13=24.0, hr_14=27.0, hr_15=18.0, hr_16=23.0, hr_17=18.0, hr_18=19.0, hr_19=18.0, hr_20=21.0, hr_21=23.0, hr_22=18.0, hr_23=19.0, hr_24=19.0, location='龍潭', measure='PM2.5'),
 Row(date='2015/01/03', hr_01=14.0, hr_02=14.0, hr_03=9.0, hr_04=13.0, hr_05=12.0, hr_06=15.0, hr_07=9.0, hr_08=11.0, hr_09=14.0, hr_10=27.0, hr_11=27.0, hr_12=24.0, hr_13=16.0, hr_14=27.0, hr_15=38.0, hr_16=39.0, hr_17=35.0, hr_18=32.0, hr_19=38.0, hr_20=36.0, hr_21=34.0, hr_22=36.0, hr_23=36.0, hr_24=39.0, l

In [0]:
df = spark.createDataFrame(dalipm25row)
df.show()

+----------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+--------+-------+
|      date|hr_01|hr_02|hr_03|hr_04|hr_05|hr_06|hr_07|hr_08|hr_09|hr_10|hr_11|hr_12|hr_13|hr_14|hr_15|hr_16|hr_17|hr_18|hr_19|hr_20|hr_21|hr_22|hr_23|hr_24|location|measure|
+----------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+--------+-------+
|2015/01/01| 46.0| 71.0| 76.0| 74.0| 65.0| 62.0| 56.0| 50.0| 52.0| 56.0| 54.0| 47.0| 40.0| 36.0| 37.0| 27.0| 30.0| 25.0| 26.0| 24.0| 18.0| 16.0| 11.0| 14.0|      龍潭|  PM2.5|
|2015/01/02| 15.0| 12.0|  9.0| 14.0| 17.0| 20.0| 18.0| 22.0| 21.0| 23.0| 18.0| 25.0| 24.0| 27.0| 18.0| 23.0| 18.0| 19.0| 18.0| 21.0| 23.0| 18.0| 19.0| 19.0|      龍潭|  PM2.5|
|2015/01/03| 14.0| 14.0|  9.0| 13.0| 12.0| 15.0|  9.0| 11.0| 14.0| 27.0| 27.0| 24.0| 16.0| 27.0| 38.0| 39.0| 35.0| 32.0| 38.0| 36.

In [0]:
df.describe().show()

+-------+----------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+-----------------+------------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+------------------+-----------------+------------------+------------------+--------+-------+
|summary|      date|             hr_01|             hr_02|             hr_03|             hr_04|            hr_05|             hr_06|             hr_07|            hr_08|             hr_09|             hr_10|            hr_11|            hr_12|             hr_13|             hr_14|             hr_15|             hr_16|             hr_17|             hr_18|             hr_19|            hr_20|             hr_21|            hr_22|             hr_23|             hr_24|location|measure|
+-------+----------+----

In [0]:
df.filter(df["date"]=="2015/01/22").select("hr_01","location").show()

+-----+--------+
|hr_01|location|
+-----+--------+
| 36.0|      龍潭|
| 48.0|      麥寮|
| 41.0|      馬祖|
| 36.0|      馬公|
| 39.0|      頭份|
| 26.0|      陽明|
| 26.0|      關山|
| 71.0|      金門|
| 37.0|      觀音|
| 21.0|      萬里|
| 30.0|      萬華|
| 42.0|      苗栗|
| 21.0|      花蓮|
| 39.0|      臺西|
| 26.0|      臺東|
| 53.0|      臺南|
| 45.0|      美濃|
| 48.0|      線西|
| 28.0|      竹東|
| 56.0|      潮州|
+-----+--------+
only showing top 20 rows



# 直接透過rdd生成data frame. toDF()

In [0]:
dalipm25.toDF().show()

+----------+---+-----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|        _1| _2|   _3| _4| _5| _6| _7| _8| _9|_10|_11|_12|_13|_14|_15|_16|_17|_18|_19|_20|_21|_22|_23|_24|_25|_26|_27|
+----------+---+-----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|2015/01/01| 龍潭|PM2.5| 46| 71| 76| 74| 65| 62| 56| 50| 52| 56| 54| 47| 40| 36| 37| 27| 30| 25| 26| 24| 18| 16| 11| 14|
|2015/01/02| 龍潭|PM2.5| 15| 12|  9| 14| 17| 20| 18| 22| 21| 23| 18| 25| 24| 27| 18| 23| 18| 19| 18| 21| 23| 18| 19| 19|
|2015/01/03| 龍潭|PM2.5| 14| 14|  9| 13| 12| 15|  9| 11| 14| 27| 27| 24| 16| 27| 38| 39| 35| 32| 38| 36| 34| 36| 36| 39|
|2015/01/04| 龍潭|PM2.5| 42| 40| 39| 34| 33| 25| 24| 23| 31| 27| 34| 35| 35| 38| 46| 59| 53| 58| 61| 74| 80| 78| 70| 53|
|2015/01/05| 龍潭|PM2.5| 42| 33| 24| 21| 20| 18| 16| 21| 21| 17| 13| 23| 36| 49| 61| 63| 68| 81| 91| 99| 76| 57| 34| 24|
|2015/01/06| 龍潭|PM2.5| 19| 13| 11| 16| 11|  9|  

In [0]:
dalipm25.toDF(pm25schema).printSchema()

root
 |-- 日期: string (nullable = true)
 |-- 測站: string (nullable = true)
 |-- 測項: string (nullable = true)
 |-- 00: string (nullable = true)
 |-- 01: string (nullable = true)
 |-- 02: string (nullable = true)
 |-- 03: string (nullable = true)
 |-- 04: string (nullable = true)
 |-- 05: string (nullable = true)
 |-- 06: string (nullable = true)
 |-- 07: string (nullable = true)
 |-- 08: string (nullable = true)
 |-- 09: string (nullable = true)
 |-- 10: string (nullable = true)
 |-- 11: string (nullable = true)
 |-- 12: string (nullable = true)
 |-- 13: string (nullable = true)
 |-- 14: string (nullable = true)
 |-- 15: string (nullable = true)
 |-- 16: string (nullable = true)
 |-- 17: string (nullable = true)
 |-- 18: string (nullable = true)
 |-- 19: string (nullable = true)
 |-- 20: string (nullable = true)
 |-- 21: string (nullable = true)
 |-- 22: string (nullable = true)
 |-- 23: string (nullable = true)



In [0]:
pm25schema = weather_data_rdd.first()
dalipm25.toDF(pm25schema).show()

+----------+---+-----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|        日期| 測站|   測項| 00| 01| 02| 03| 04| 05| 06| 07| 08| 09| 10| 11| 12| 13| 14| 15| 16| 17| 18| 19| 20| 21| 22| 23|
+----------+---+-----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|2015/01/01| 龍潭|PM2.5| 46| 71| 76| 74| 65| 62| 56| 50| 52| 56| 54| 47| 40| 36| 37| 27| 30| 25| 26| 24| 18| 16| 11| 14|
|2015/01/02| 龍潭|PM2.5| 15| 12|  9| 14| 17| 20| 18| 22| 21| 23| 18| 25| 24| 27| 18| 23| 18| 19| 18| 21| 23| 18| 19| 19|
|2015/01/03| 龍潭|PM2.5| 14| 14|  9| 13| 12| 15|  9| 11| 14| 27| 27| 24| 16| 27| 38| 39| 35| 32| 38| 36| 34| 36| 36| 39|
|2015/01/04| 龍潭|PM2.5| 42| 40| 39| 34| 33| 25| 24| 23| 31| 27| 34| 35| 35| 38| 46| 59| 53| 58| 61| 74| 80| 78| 70| 53|
|2015/01/05| 龍潭|PM2.5| 42| 33| 24| 21| 20| 18| 16| 21| 21| 17| 13| 23| 36| 49| 61| 63| 68| 81| 91| 99| 76| 57| 34| 24|
|2015/01/06| 龍潭|PM2.5| 19| 13| 11| 16| 11|  9|  

In [0]:
dfpm25 = dalipm25.toDF(pm25schema)

# The code below is string, so it is sort by string (0 smallest, 9 biggest)

In [0]:
dfpm25.describe().show()

+-------+----------+-----+-----+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+-----------------+------------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+------------------+-----------------+------------------+------------------+
|summary|        日期|   測站|   測項|                00|                01|                02|                03|               04|                05|                06|               07|                08|                09|               10|               11|                12|                13|                14|                15|                16|                17|                18|               19|                20|               21|                22|                23|
+-------+----------+-----+-----+--

In [0]:
dfpm25.columns

['日期',
 '測站',
 '測項',
 '00',
 '01',
 '02',
 '03',
 '04',
 '05',
 '06',
 '07',
 '08',
 '09',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23']

In [0]:
for i in dfpm25.columns[3:]:
  dfpm25 = dfpm25.withColumn(i, dfpm25[i].cast("double"))

In [0]:
dfpm25.printSchema()

root
 |-- 日期: string (nullable = true)
 |-- 測站: string (nullable = true)
 |-- 測項: string (nullable = true)
 |-- 00: double (nullable = true)
 |-- 01: double (nullable = true)
 |-- 02: double (nullable = true)
 |-- 03: double (nullable = true)
 |-- 04: double (nullable = true)
 |-- 05: double (nullable = true)
 |-- 06: double (nullable = true)
 |-- 07: double (nullable = true)
 |-- 08: double (nullable = true)
 |-- 09: double (nullable = true)
 |-- 10: double (nullable = true)
 |-- 11: double (nullable = true)
 |-- 12: double (nullable = true)
 |-- 13: double (nullable = true)
 |-- 14: double (nullable = true)
 |-- 15: double (nullable = true)
 |-- 16: double (nullable = true)
 |-- 17: double (nullable = true)
 |-- 18: double (nullable = true)
 |-- 19: double (nullable = true)
 |-- 20: double (nullable = true)
 |-- 21: double (nullable = true)
 |-- 22: double (nullable = true)
 |-- 23: double (nullable = true)



In [0]:
dfpm25.describe().show()

+-------+----------+-----+-----+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+-----------------+------------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+------------------+-----------------+------------------+------------------+
|summary|        日期|   測站|   測項|                00|                01|                02|                03|               04|                05|                06|               07|                08|                09|               10|               11|                12|                13|                14|                15|                16|                17|                18|               19|                20|               21|                22|                23|
+-------+----------+-----+-----+--

# 使用 DataFrame.filter() 來進行row資料之條件計算 http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame

In [0]:
df.filter(df.hr_01>30).show()

+----------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+--------+-------+
|      date|hr_01|hr_02|hr_03|hr_04|hr_05|hr_06|hr_07|hr_08|hr_09|hr_10|hr_11|hr_12|hr_13|hr_14|hr_15|hr_16|hr_17|hr_18|hr_19|hr_20|hr_21|hr_22|hr_23|hr_24|location|measure|
+----------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+--------+-------+
|2015/01/01| 46.0| 71.0| 76.0| 74.0| 65.0| 62.0| 56.0| 50.0| 52.0| 56.0| 54.0| 47.0| 40.0| 36.0| 37.0| 27.0| 30.0| 25.0| 26.0| 24.0| 18.0| 16.0| 11.0| 14.0|      龍潭|  PM2.5|
|2015/01/04| 42.0| 40.0| 39.0| 34.0| 33.0| 25.0| 24.0| 23.0| 31.0| 27.0| 34.0| 35.0| 35.0| 38.0| 46.0| 59.0| 53.0| 58.0| 61.0| 74.0| 80.0| 78.0| 70.0| 53.0|      龍潭|  PM2.5|
|2015/01/05| 42.0| 33.0| 24.0| 21.0| 20.0| 18.0| 16.0| 21.0| 21.0| 17.0| 13.0| 23.0| 36.0| 49.0| 61.0| 63.0| 68.0| 81.0| 91.0| 99.

# 使用 DataFrame.select() 來進行資料之Projection http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame

In [0]:
df.select("hr_01", "hr_02", "location", "measure").show()

+-----+-----+--------+-------+
|hr_01|hr_02|location|measure|
+-----+-----+--------+-------+
| 46.0| 71.0|      龍潭|  PM2.5|
| 15.0| 12.0|      龍潭|  PM2.5|
| 14.0| 14.0|      龍潭|  PM2.5|
| 42.0| 40.0|      龍潭|  PM2.5|
| 42.0| 33.0|      龍潭|  PM2.5|
| 19.0| 13.0|      龍潭|  PM2.5|
| 22.0| 30.0|      龍潭|  PM2.5|
| 23.0| 23.0|      龍潭|  PM2.5|
| 20.0| 18.0|      龍潭|  PM2.5|
| 31.0| 30.0|      龍潭|  PM2.5|
| 11.0| 12.0|      龍潭|  PM2.5|
|  3.0|  5.0|      龍潭|  PM2.5|
| 14.0|  8.0|      龍潭|  PM2.5|
|  8.0| 14.0|      龍潭|  PM2.5|
| 23.0| 33.0|      龍潭|  PM2.5|
| 55.0| 51.0|      龍潭|  PM2.5|
| 12.0| 11.0|      龍潭|  PM2.5|
| 34.0| 41.0|      龍潭|  PM2.5|
| 34.0| 39.0|      龍潭|  PM2.5|
| 36.0| 34.0|      龍潭|  PM2.5|
+-----+-----+--------+-------+
only showing top 20 rows



## 使用 DataFrame.describe() 來進行DataFrame or Column 資料之統計 http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame

In [0]:
df.describe().show()

+-------+----------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+-----------------+------------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+------------------+-----------------+------------------+------------------+--------+-------+
|summary|      date|             hr_01|             hr_02|             hr_03|             hr_04|            hr_05|             hr_06|             hr_07|            hr_08|             hr_09|             hr_10|            hr_11|            hr_12|             hr_13|             hr_14|             hr_15|             hr_16|             hr_17|             hr_18|             hr_19|            hr_20|             hr_21|            hr_22|             hr_23|             hr_24|location|measure|
+-------+----------+----

# 使用 DataFrame.agg() 來進行column數值之統計計算 http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame

In [0]:
df.groupBy().avg().show()

+------------------+------------------+-----------------+------------------+-----------------+------------------+------------------+-----------------+------------------+------------------+-----------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+------------------+-----------------+------------------+------------------+
|        avg(hr_01)|        avg(hr_02)|       avg(hr_03)|        avg(hr_04)|       avg(hr_05)|        avg(hr_06)|        avg(hr_07)|       avg(hr_08)|        avg(hr_09)|        avg(hr_10)|       avg(hr_11)|       avg(hr_12)|       avg(hr_13)|        avg(hr_14)|        avg(hr_15)|        avg(hr_16)|        avg(hr_17)|        avg(hr_18)|        avg(hr_19)|       avg(hr_20)|        avg(hr_21)|       avg(hr_22)|        avg(hr_23)|        avg(hr_24)|
+------------------+------------------+-----------------+------------------+-----------------+------

In [0]:
from pyspark.sql import functions as F
df.agg(F.mean(df.hr_01),F.mean(df.hr_02)).show()

+------------------+------------------+
|        avg(hr_01)|        avg(hr_02)|
+------------------+------------------+
|21.535227715064227|21.192244063838068|
+------------------+------------------+



#使用 DataFrame.withColumn() 來進行column新增column http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame

In [0]:
df.show()

+----------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+--------+-------+
|      date|hr_01|hr_02|hr_03|hr_04|hr_05|hr_06|hr_07|hr_08|hr_09|hr_10|hr_11|hr_12|hr_13|hr_14|hr_15|hr_16|hr_17|hr_18|hr_19|hr_20|hr_21|hr_22|hr_23|hr_24|location|measure|
+----------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+--------+-------+
|2015/01/01| 46.0| 71.0| 76.0| 74.0| 65.0| 62.0| 56.0| 50.0| 52.0| 56.0| 54.0| 47.0| 40.0| 36.0| 37.0| 27.0| 30.0| 25.0| 26.0| 24.0| 18.0| 16.0| 11.0| 14.0|      龍潭|  PM2.5|
|2015/01/02| 15.0| 12.0|  9.0| 14.0| 17.0| 20.0| 18.0| 22.0| 21.0| 23.0| 18.0| 25.0| 24.0| 27.0| 18.0| 23.0| 18.0| 19.0| 18.0| 21.0| 23.0| 18.0| 19.0| 19.0|      龍潭|  PM2.5|
|2015/01/03| 14.0| 14.0|  9.0| 13.0| 12.0| 15.0|  9.0| 11.0| 14.0| 27.0| 27.0| 24.0| 16.0| 27.0| 38.0| 39.0| 35.0| 32.0| 38.0| 36.

In [0]:
df.withColumn('hr_avg', df.hr_01+df.hr_02).show()

+----------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+--------+-------+------+
|      date|hr_01|hr_02|hr_03|hr_04|hr_05|hr_06|hr_07|hr_08|hr_09|hr_10|hr_11|hr_12|hr_13|hr_14|hr_15|hr_16|hr_17|hr_18|hr_19|hr_20|hr_21|hr_22|hr_23|hr_24|location|measure|hr_avg|
+----------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+--------+-------+------+
|2015/01/01| 46.0| 71.0| 76.0| 74.0| 65.0| 62.0| 56.0| 50.0| 52.0| 56.0| 54.0| 47.0| 40.0| 36.0| 37.0| 27.0| 30.0| 25.0| 26.0| 24.0| 18.0| 16.0| 11.0| 14.0|      龍潭|  PM2.5| 117.0|
|2015/01/02| 15.0| 12.0|  9.0| 14.0| 17.0| 20.0| 18.0| 22.0| 21.0| 23.0| 18.0| 25.0| 24.0| 27.0| 18.0| 23.0| 18.0| 19.0| 18.0| 21.0| 23.0| 18.0| 19.0| 19.0|      龍潭|  PM2.5|  27.0|
|2015/01/03| 14.0| 14.0|  9.0| 13.0| 12.0| 15.0|  9.0| 11.0| 14.0| 27.0| 27.0| 24.0| 16.0| 27.0

# 練習1: 請算算看2015全國哪個測站，紫爆天數最多？

# 練習2: 請使用SPARK SQL求取2015年，全國pm2.5最高的前十個工作站測點以及其日期。

In [0]:
df = clean_weather_data.toDF()

In [0]:
df.filter(df["_3"]=="PM2.5").show()

+----------+---+-----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|        _1| _2|   _3| _4| _5| _6| _7| _8| _9|_10|_11|_12|_13|_14|_15|_16|_17|_18|_19|_20|_21|_22|_23|_24|_25|_26|_27|
+----------+---+-----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|2015/01/01| 龍潭|PM2.5| 46| 71| 76| 74| 65| 62| 56| 50| 52| 56| 54| 47| 40| 36| 37| 27| 30| 25| 26| 24| 18| 16| 11| 14|
|2015/01/02| 龍潭|PM2.5| 15| 12|  9| 14| 17| 20| 18| 22| 21| 23| 18| 25| 24| 27| 18| 23| 18| 19| 18| 21| 23| 18| 19| 19|
|2015/01/03| 龍潭|PM2.5| 14| 14|  9| 13| 12| 15|  9| 11| 14| 27| 27| 24| 16| 27| 38| 39| 35| 32| 38| 36| 34| 36| 36| 39|
|2015/01/04| 龍潭|PM2.5| 42| 40| 39| 34| 33| 25| 24| 23| 31| 27| 34| 35| 35| 38| 46| 59| 53| 58| 61| 74| 80| 78| 70| 53|
|2015/01/05| 龍潭|PM2.5| 42| 33| 24| 21| 20| 18| 16| 21| 21| 17| 13| 23| 36| 49| 61| 63| 68| 81| 91| 99| 76| 57| 34| 24|
|2015/01/06| 龍潭|PM2.5| 19| 13| 11| 16| 11|  9|  

In [0]:
for i in df.columns[3:]:
  df = df.withColumn(i, df[i].cast("double"))

In [0]:
df.filter(df["_3"]=="PM2.5").withColumn("dayavg", 
                                        df["_4"]+df["_5"]+df["_6"]+df["_7"]+df["_8"]+df["_9"]+df["_10"]+df["_11"]\
                                        +df["_12"]+df["_13"]+df["_14"]+df["_15"]+df["_16"]+df["_17"]+df["_18"]+df["_19"]\
                                        +df["_20"]+df["_21"]+df["_22"]+df["_23"]+df["_24"]+df["_25"]+df["_26"]+df["_27"]\
                                       ).show()

+----------+---+-----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+------+
|        _1| _2|   _3|  _4|  _5|  _6|  _7|  _8|  _9| _10| _11| _12| _13| _14| _15| _16| _17| _18| _19| _20| _21| _22| _23| _24| _25| _26| _27|dayavg|
+----------+---+-----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+------+
|2015/01/01| 龍潭|PM2.5|46.0|71.0|76.0|74.0|65.0|62.0|56.0|50.0|52.0|56.0|54.0|47.0|40.0|36.0|37.0|27.0|30.0|25.0|26.0|24.0|18.0|16.0|11.0|14.0|1013.0|
|2015/01/02| 龍潭|PM2.5|15.0|12.0| 9.0|14.0|17.0|20.0|18.0|22.0|21.0|23.0|18.0|25.0|24.0|27.0|18.0|23.0|18.0|19.0|18.0|21.0|23.0|18.0|19.0|19.0| 461.0|
|2015/01/03| 龍潭|PM2.5|14.0|14.0| 9.0|13.0|12.0|15.0| 9.0|11.0|14.0|27.0|27.0|24.0|16.0|27.0|38.0|39.0|35.0|32.0|38.0|36.0|34.0|36.0|36.0|39.0| 595.0|
|2015/01/04| 龍潭|PM2.5|42.0|40.0|39.0|34.0|33.0|25.0|24.0|23.0|31.0|27.0|34.0|35.0|35.0|38.0|46.0|59.

In [0]:
df.filter(df["_3"]=="PM2.5").withColumn("dayavg", 
                                        df["_4"]+df["_5"]+df["_6"]+df["_7"]+df["_8"]+df["_9"]+df["_10"]+df["_11"]\
                                        +df["_12"]+df["_13"]+df["_14"]+df["_15"]+df["_16"]+df["_17"]+df["_18"]+df["_19"]\
                                        +df["_20"]+df["_21"]+df["_22"]+df["_23"]+df["_24"]+df["_25"]+df["_26"]+df["_27"]\
                                       ).show()

+----------+---+-----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+------+
|        _1| _2|   _3|  _4|  _5|  _6|  _7|  _8|  _9| _10| _11| _12| _13| _14| _15| _16| _17| _18| _19| _20| _21| _22| _23| _24| _25| _26| _27|dayavg|
+----------+---+-----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+------+
|2015/01/01| 龍潭|PM2.5|46.0|71.0|76.0|74.0|65.0|62.0|56.0|50.0|52.0|56.0|54.0|47.0|40.0|36.0|37.0|27.0|30.0|25.0|26.0|24.0|18.0|16.0|11.0|14.0|1013.0|
|2015/01/02| 龍潭|PM2.5|15.0|12.0| 9.0|14.0|17.0|20.0|18.0|22.0|21.0|23.0|18.0|25.0|24.0|27.0|18.0|23.0|18.0|19.0|18.0|21.0|23.0|18.0|19.0|19.0| 461.0|
|2015/01/03| 龍潭|PM2.5|14.0|14.0| 9.0|13.0|12.0|15.0| 9.0|11.0|14.0|27.0|27.0|24.0|16.0|27.0|38.0|39.0|35.0|32.0|38.0|36.0|34.0|36.0|36.0|39.0| 595.0|
|2015/01/04| 龍潭|PM2.5|42.0|40.0|39.0|34.0|33.0|25.0|24.0|23.0|31.0|27.0|34.0|35.0|35.0|38.0|46.0|59.

In [0]:
avgdf = df.filter(df["_3"]=="PM2.5").withColumn("dayavg", 
                                        (
                                        df["_4"]+df["_5"]+df["_6"]+df["_7"]+df["_8"]+df["_9"]+df["_10"]+df["_11"]\
                                        +df["_12"]+df["_13"]+df["_14"]+df["_15"]+df["_16"]+df["_17"]+df["_18"]+df["_19"]\
                                        +df["_20"]+df["_21"]+df["_22"]+df["_23"]+df["_24"]+df["_25"]+df["_26"]+df["_27"])/24)\
                                        .select("_1", "_2","dayavg")

In [0]:
avgdf.show()

+----------+---+------------------+
|        _1| _2|            dayavg|
+----------+---+------------------+
|2015/01/01| 龍潭|42.208333333333336|
|2015/01/02| 龍潭|19.208333333333332|
|2015/01/03| 龍潭|24.791666666666668|
|2015/01/04| 龍潭|              45.5|
|2015/01/05| 龍潭|              42.0|
|2015/01/06| 龍潭|            27.375|
|2015/01/08| 龍潭|37.166666666666664|
|2015/01/09| 龍潭|              24.0|
|2015/01/10| 龍潭|            16.875|
|2015/01/11| 龍潭|29.708333333333332|
|2015/01/13| 龍潭|20.541666666666668|
|2015/01/14| 龍潭|10.041666666666666|
|2015/01/15| 龍潭|             16.75|
|2015/01/16| 龍潭|21.708333333333332|
|2015/01/17| 龍潭|58.041666666666664|
|2015/01/18| 龍潭|25.833333333333332|
|2015/01/19| 龍潭|            32.125|
|2015/01/20| 龍潭|36.333333333333336|
|2015/01/21| 龍潭|36.666666666666664|
|2015/01/22| 龍潭|            50.125|
+----------+---+------------------+
only showing top 20 rows



In [0]:
avgdf.filter(avgdf.dayavg>60).show()

+----------+---+------------------+
|        _1| _2|            dayavg|
+----------+---+------------------+
|2015/02/05| 龍潭| 68.95833333333333|
|2015/12/16| 龍潭| 74.54166666666667|
|2015/01/04| 麥寮|60.166666666666664|
|2015/01/05| 麥寮|              79.5|
|2015/01/17| 麥寮| 65.29166666666667|
|2015/01/24| 麥寮| 80.91666666666667|
|2015/01/25| 麥寮| 71.58333333333333|
|2015/02/05| 麥寮| 75.66666666666667|
|2015/02/14| 麥寮|61.541666666666664|
|2015/02/15| 麥寮| 75.29166666666667|
|2015/03/14| 麥寮|             63.75|
|2015/03/15| 麥寮| 74.33333333333333|
|2015/03/20| 麥寮| 67.79166666666667|
|2015/12/16| 麥寮|             71.25|
|2015/01/17| 鳳山|             61.25|
|2015/01/18| 鳳山|60.458333333333336|
|2015/01/25| 鳳山|              60.5|
|2015/01/26| 鳳山| 67.70833333333333|
|2015/02/05| 鳳山|             62.75|
|2015/02/06| 鳳山| 79.04166666666667|
+----------+---+------------------+
only showing top 20 rows



In [0]:
avgdf.filter(avgdf.dayavg>60).groupby(avgdf._2).count().show()

+---+-----+
| _2|count|
+---+-----+
| 頭份|    4|
| 金門|   21|
| 新港|    8|
| 崙背|   23|
| 彰化|    6|
| 龍潭|    2|
| 新竹|    1|
| 竹山|   28|
| 松山|    2|
| 善化|   18|
| 楠梓|   13|
| 竹東|    2|
| 土城|    1|
| 小港|   11|
| 臺西|   10|
| 潮州|    7|
| 苗栗|    3|
| 仁武|   10|
| 馬祖|    9|
| 西屯|    9|
+---+-----+
only showing top 20 rows



In [0]:
avgdf.filter(avgdf.dayavg>60).groupby(avgdf._2).count().orderBy('count', ascending=False).show()

+---+-----+
| _2|count|
+---+-----+
| 竹山|   28|
| 斗六|   26|
| 崙背|   23|
| 金門|   21|
| 善化|   18|
| 埔里|   16|
| 橋頭|   14|
| 嘉義|   14|
| 左營|   14|
| 楠梓|   13|
| 復興|   13|
| 麥寮|   12|
| 忠明|   11|
| 鳳山|   11|
| 小港|   11|
| 大寮|   11|
| 臺西|   10|
| 二林|   10|
| 大里|   10|
| 仁武|   10|
+---+-----+
only showing top 20 rows



# 使用 Spark SQL來下達SQL查詢

In [0]:
df = spark.createDataFrame(dalipm25row)
df.show()
df.registerTempTable("Table")

+----------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+--------+-------+
|      date|hr_01|hr_02|hr_03|hr_04|hr_05|hr_06|hr_07|hr_08|hr_09|hr_10|hr_11|hr_12|hr_13|hr_14|hr_15|hr_16|hr_17|hr_18|hr_19|hr_20|hr_21|hr_22|hr_23|hr_24|location|measure|
+----------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+--------+-------+
|2015/01/01| 46.0| 71.0| 76.0| 74.0| 65.0| 62.0| 56.0| 50.0| 52.0| 56.0| 54.0| 47.0| 40.0| 36.0| 37.0| 27.0| 30.0| 25.0| 26.0| 24.0| 18.0| 16.0| 11.0| 14.0|      龍潭|  PM2.5|
|2015/01/02| 15.0| 12.0|  9.0| 14.0| 17.0| 20.0| 18.0| 22.0| 21.0| 23.0| 18.0| 25.0| 24.0| 27.0| 18.0| 23.0| 18.0| 19.0| 18.0| 21.0| 23.0| 18.0| 19.0| 19.0|      龍潭|  PM2.5|
|2015/01/03| 14.0| 14.0|  9.0| 13.0| 12.0| 15.0|  9.0| 11.0| 14.0| 27.0| 27.0| 24.0| 16.0| 27.0| 38.0| 39.0| 35.0| 32.0| 38.0| 36.

In [0]:
spark.sql("""
            select * from Table where date ='2015/02/07'
          """).show()

+----------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+--------+-------+
|      date|hr_01|hr_02|hr_03|hr_04|hr_05|hr_06|hr_07|hr_08|hr_09|hr_10|hr_11|hr_12|hr_13|hr_14|hr_15|hr_16|hr_17|hr_18|hr_19|hr_20|hr_21|hr_22|hr_23|hr_24|location|measure|
+----------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+--------+-------+
|2015/02/07| 14.0| 16.0| 15.0|  8.0|  7.0|  9.0| 10.0| 13.0| 17.0| 15.0| 18.0| 15.0| 32.0| 33.0| 36.0| 20.0| 18.0| 17.0| 25.0| 25.0| 22.0| 18.0| 17.0| 16.0|      龍潭|  PM2.5|
|2015/02/07| 31.0| 30.0| 36.0| 39.0| 35.0| 35.0| 35.0| 35.0| 34.0| 36.0| 31.0| 31.0| 49.0| 56.0| 44.0| 37.0| 40.0| 35.0| 30.0| 29.0| 24.0| 24.0| 22.0| 22.0|      麥寮|  PM2.5|
|2015/02/07| 41.0| 44.0| 43.0| 51.0| 49.0| 59.0| 69.0| 75.0| 75.0| 75.0| 80.0| 76.0| 75.0| 68.0| 55.0| 53.0| 39.0| 46.0| 36.0| 48.

In [0]:
s = spark.sql("""
                select count(*) count 
                from Table 
                where hr_01 > 100
               """)

In [0]:
type(s)

pyspark.sql.dataframe.DataFrame

In [0]:
spark.sql("""
                select date, location 
                from Table
                where date="2015/02/07"
          """).show()

+----------+--------+
|      date|location|
+----------+--------+
|2015/02/07|      龍潭|
|2015/02/07|      麥寮|
|2015/02/07|      鳳山|
|2015/02/07|      馬祖|
|2015/02/07|      馬公|
|2015/02/07|      頭份|
|2015/02/07|      陽明|
|2015/02/07|      關山|
|2015/02/07|      金門|
|2015/02/07|      觀音|
|2015/02/07|      西屯|
|2015/02/07|      萬里|
|2015/02/07|      萬華|
|2015/02/07|      菜寮|
|2015/02/07|      苗栗|
|2015/02/07|      花蓮|
|2015/02/07|      臺西|
|2015/02/07|      臺東|
|2015/02/07|      臺南|
|2015/02/07|      美濃|
+----------+--------+
only showing top 20 rows



In [0]:
spark.sql("""
                select date,location, hr_01, hr_02, hr_01-hr_02 as diff 
                from Table
          """).show()

+----------+--------+-----+-----+-----+
|      date|location|hr_01|hr_02| diff|
+----------+--------+-----+-----+-----+
|2015/01/01|      龍潭| 46.0| 71.0|-25.0|
|2015/01/02|      龍潭| 15.0| 12.0|  3.0|
|2015/01/03|      龍潭| 14.0| 14.0|  0.0|
|2015/01/04|      龍潭| 42.0| 40.0|  2.0|
|2015/01/05|      龍潭| 42.0| 33.0|  9.0|
|2015/01/06|      龍潭| 19.0| 13.0|  6.0|
|2015/01/08|      龍潭| 22.0| 30.0| -8.0|
|2015/01/09|      龍潭| 23.0| 23.0|  0.0|
|2015/01/10|      龍潭| 20.0| 18.0|  2.0|
|2015/01/11|      龍潭| 31.0| 30.0|  1.0|
|2015/01/13|      龍潭| 11.0| 12.0| -1.0|
|2015/01/14|      龍潭|  3.0|  5.0| -2.0|
|2015/01/15|      龍潭| 14.0|  8.0|  6.0|
|2015/01/16|      龍潭|  8.0| 14.0| -6.0|
|2015/01/17|      龍潭| 23.0| 33.0|-10.0|
|2015/01/18|      龍潭| 55.0| 51.0|  4.0|
|2015/01/19|      龍潭| 12.0| 11.0|  1.0|
|2015/01/20|      龍潭| 34.0| 41.0| -7.0|
|2015/01/21|      龍潭| 34.0| 39.0| -5.0|
|2015/01/22|      龍潭| 36.0| 34.0|  2.0|
+----------+--------+-----+-----+-----+
only showing top 20 rows



## 練習1: 請使用SparkSQL，算算看2015全國哪個測站，紫爆天數最多？

In [0]:
expr=""
for i in df.columns[1:25]:
  expr=str(i)+"+"+expr

print(expr)

hr_24+hr_23+hr_22+hr_21+hr_20+hr_19+hr_18+hr_17+hr_16+hr_15+hr_14+hr_13+hr_12+hr_11+hr_10+hr_09+hr_08+hr_07+hr_06+hr_05+hr_04+hr_03+hr_02+hr_01+


In [0]:
spark.sql("""
                select location, date, (hr_24+hr_23+hr_22+hr_21+hr_20+hr_19+hr_18+hr_17+hr_16+hr_15+hr_14+hr_13+hr_12+hr_11+hr_10+hr_09+hr_08+hr_07+hr_06+hr_05+hr_04+hr_03+hr_02+hr_01)/24 as avg
                from Table 
                having avg>60
          """).registerTempTable("DayAvgTable")

spark.sql("""
          select * 
          from DayAvgTable
          """).show()

+--------+----------+------------------+
|location|      date|               avg|
+--------+----------+------------------+
|      龍潭|2015/02/05| 68.95833333333333|
|      龍潭|2015/12/16| 74.54166666666667|
|      麥寮|2015/01/04|60.166666666666664|
|      麥寮|2015/01/05|              79.5|
|      麥寮|2015/01/17| 65.29166666666667|
|      麥寮|2015/01/24| 80.91666666666667|
|      麥寮|2015/01/25| 71.58333333333333|
|      麥寮|2015/02/05| 75.66666666666667|
|      麥寮|2015/02/14|61.541666666666664|
|      麥寮|2015/02/15| 75.29166666666667|
|      麥寮|2015/03/14|             63.75|
|      麥寮|2015/03/15| 74.33333333333333|
|      麥寮|2015/03/20| 67.79166666666667|
|      麥寮|2015/12/16|             71.25|
|      鳳山|2015/01/17|             61.25|
|      鳳山|2015/01/18|60.458333333333336|
|      鳳山|2015/01/25|              60.5|
|      鳳山|2015/01/26| 67.70833333333333|
|      鳳山|2015/02/05|             62.75|
|      鳳山|2015/02/06| 79.04166666666667|
+--------+----------+------------------+
only showing top

In [0]:
spark.sql("""
          select location, count(date) as count
          from DayAvgTable
          group by location
          order by count DESC
          """).show()

+--------+-----+
|location|count|
+--------+-----+
|      竹山|   28|
|      斗六|   26|
|      崙背|   23|
|      金門|   21|
|      善化|   18|
|      埔里|   16|
|      橋頭|   14|
|      嘉義|   14|
|      左營|   14|
|      復興|   13|
|      楠梓|   13|
|      麥寮|   12|
|      忠明|   11|
|      鳳山|   11|
|      小港|   11|
|      大寮|   11|
|      臺西|   10|
|      二林|   10|
|      大里|   10|
|      仁武|   10|
+--------+-----+
only showing top 20 rows



# 利用DataFrame 直接讀取CSV, Fillna, DropNa功能清洗資料

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .getOrCreate()

from pyspark.sql import SQLContext
from pyspark.sql import Row

In [0]:
df = spark.read.load("./pm25.csv", format="csv")

In [0]:
df.show(200)

+----------+---+----------+----+-----+----+----+----+-----+----+----+----+----+----+----+----+----+-----+----+----+----+----+----+----+----+-----+-----+
|       _c0|_c1|       _c2| _c3|  _c4| _c5| _c6| _c7|  _c8| _c9|_c10|_c11|_c12|_c13|_c14|_c15|_c16| _c17|_c18|_c19|_c20|_c21|_c22|_c23|_c24| _c25| _c26|
+----------+---+----------+----+-----+----+----+----+-----+----+----+----+----+----+----+----+----+-----+----+----+----+----+----+----+----+-----+-----+
|        日期| 測站|        測項|  00|   01|  02|  03|  04|   05|  06|  07|  08|  09|  10|  11|  12|  13|   14|  15|  16|  17|  18|  19|  20|  21|   22|   23|
|2015/01/01| 龍潭|  AMB_TEMP|  14|   14|  14|  13|  13|   13|  12|  12|  13|  14|  14|  14|  14|  14|   13|  13|  12|  11|  11|  11|  11|  11|   11|   11|
|2015/01/01| 龍潭|        CO|0.69| 0.72|0.69|0.64|0.54| 0.47|0.45|0.48|0.51|0.54|0.54| 0.5|0.47|0.38| 0.36|0.35|0.34|0.37|0.34|0.29|0.26|0.22| 0.19| 0.18|
|2015/01/01| 龍潭|        NO| 0.3|  0.1| 0.6|   2|   2|  1.9| 2.2| 3.1| 3.7| 4.3| 4.

In [0]:
df2=df.filter(df["_c0"]!="日期") 

In [0]:
df2.count()

465317

In [0]:
df2.dropna()

DataFrame[_c0: string, _c1: string, _c2: string, _c3: string, _c4: string, _c5: string, _c6: string, _c7: string, _c8: string, _c9: string, _c10: string, _c11: string, _c12: string, _c13: string, _c14: string, _c15: string, _c16: string, _c17: string, _c18: string, _c19: string, _c20: string, _c21: string, _c22: string, _c23: string, _c24: string, _c25: string, _c26: string]

In [0]:
df2.count()

465317

In [0]:
df2.filter(df2._c3.isNotNull()).show()

+----------+---+----------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|       _c0|_c1|       _c2| _c3| _c4| _c5| _c6| _c7| _c8| _c9|_c10|_c11|_c12|_c13|_c14|_c15|_c16|_c17|_c18|_c19|_c20|_c21|_c22|_c23|_c24|_c25|_c26|
+----------+---+----------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|2015/01/01| 龍潭|  AMB_TEMP|  14|  14|  14|  13|  13|  13|  12|  12|  13|  14|  14|  14|  14|  14|  13|  13|  12|  11|  11|  11|  11|  11|  11|  11|
|2015/01/01| 龍潭|        CO|0.69|0.72|0.69|0.64|0.54|0.47|0.45|0.48|0.51|0.54|0.54| 0.5|0.47|0.38|0.36|0.35|0.34|0.37|0.34|0.29|0.26|0.22|0.19|0.18|
|2015/01/01| 龍潭|        NO| 0.3| 0.1| 0.6|   2|   2| 1.9| 2.2| 3.1| 3.7| 4.3| 4.3| 4.5| 3.3| 4.1| 3.1| 3.6| 3.6| 2.8| 2.8| 2.5| 2.2| 1.4| 2.1|   2|
|2015/01/01| 龍潭|       NO2|  11| 9.6| 8.7| 9.1| 9.6| 9.9|  11|  13|  11|  12|  12|  11|  11| 9.9| 9.9|  10|  11|

In [0]:
for i in df2.columns[3:]:
  df2 = df2.withColumn(i, df2[i].cast("double"))

In [0]:
df2.dropna().count()

366789

In [0]:
df2 = df2.filter(df2["_c2"]!="RAINFALL")

# Perform Pearson Correlation using DataFrame.corr( )
# 練習: 計算大里區pm10, pm2.5 間之關聯度 http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame
使用前面所建立的clean_weather_data rdd資料
    
    
    corr(col1, col2, method=None)
    Calculates the correlation of two columns of a DataFrame as a double value. Currently only supports the Pearson Correlation Coefficient. DataFrame.corr() and DataFrameStatFunctions.corr() are aliases of each other.

    Parameters:	
    col1 – The name of the first column
    col2 – The name of the second column
    method – The correlation method. Currently only supports “pearson”
    New in version 1.4.

In [0]:
def Generated_Measurement(x):
    date = x[0]
    location = x[1]
    measure = x[2]
    measurements_of_a_day = []
    for i, value in enumerate(x[3:]):
        measurements_of_a_day.append((date, measure, "hr_"+str(i), value))
    return measurements_of_a_day

daliData = clean_weather_data.filter(lambda x: x[1]=="大里" and (x[2] == "PM2.5" or x[2] == "PM10" ))
daliData.cache()

PythonRDD[182] at RDD at PythonRDD.scala:48

In [0]:
daliData.toDF().show()

+----------+---+-----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|        _1| _2|   _3| _4| _5| _6| _7| _8| _9|_10|_11|_12|_13|_14|_15|_16|_17|_18|_19|_20|_21|_22|_23|_24|_25|_26|_27|
+----------+---+-----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|2015/01/01| 大里|PM2.5| 53| 55| 58| 53| 43| 36| 35| 42| 55| 64| 65| 59| 52| 44| 47| 41| 43| 40| 42| 35| 28| 20| 18| 16|
|2015/01/02| 大里| PM10| 32| 28| 29| 35| 46| 49| 40| 30| 25| 39| 51| 61| 61| 56| 63| 72| 83| 91| 89| 81| 72| 71| 84| 91|
|2015/01/02| 大里|PM2.5| 21| 22| 26| 23| 20| 18| 15| 21| 21| 25| 29| 32| 34| 29| 32| 39| 51| 51| 47| 43| 43| 48| 47| 53|
|2015/01/03| 大里| PM10| 96| 87| 78| 70| 65| 61| 54| 55| 65| 83| 96| 97|100|107|109|109|104|103|105|108|123|126|123|103|
|2015/01/03| 大里|PM2.5| 48| 48| 43| 38| 37| 36| 37| 34| 37| 46| 64| 77| 83| 75| 68| 69| 64| 65| 59| 66| 71| 66| 57| 48|
|2015/01/04| 大里|PM2.5| 60| 56| 53| 43| 53| 53| 5

In [0]:
from pyspark.sql import Row

daliDataRow = clean_weather_data.filter(lambda x: x[1]=="大里" and (x[2] == "PM2.5" or x[2] == "PM10" ))\
    .flatMap(Generated_Measurement)\
    .map(lambda x: ( (x[0], x[2]), x[1], x[3] ) )\
    .groupBy(lambda x: x[0])\
    .filter(lambda x: len(x[1])==2)\
    .mapValues(lambda x: list(x))\
    .mapValues(lambda x: [x[0][1], x[0][2], x[1][1], x[1][2]])\
    .map(lambda x:[ x[0][0], x[0][1], x[1][1], x[1][3]])\
    .map(lambda x: Row(
            date = x[0],
            time = x[1],
            pm10 = float(x[2]),
            pm25 = float(x[3])
        ))
    
df = spark.createDataFrame(daliDataRow)

In [0]:
df.show(10)

+----------+----+----+-----+
|      date|pm10|pm25| time|
+----------+----+----+-----+
|2015/01/02|28.0|22.0| hr_1|
|2015/01/02|29.0|26.0| hr_2|
|2015/01/02|35.0|23.0| hr_3|
|2015/01/02|46.0|20.0| hr_4|
|2015/01/02|40.0|15.0| hr_6|
|2015/01/02|39.0|25.0| hr_9|
|2015/01/02|56.0|29.0|hr_13|
|2015/01/02|89.0|47.0|hr_18|
|2015/01/02|71.0|48.0|hr_21|
|2015/01/03|87.0|48.0| hr_1|
+----------+----+----+-----+
only showing top 10 rows



In [0]:
df.groupBy().avg().show()

+----------------+-----------------+
|       avg(pm10)|        avg(pm25)|
+----------------+-----------------+
|62.6190119760479|32.59855289421158|
+----------------+-----------------+



In [0]:
df.corr("pm10","pm25")

0.8735125761168321

In [0]:
df.describe().show()

+-------+----------+------------------+------------------+----+
|summary|      date|              pm10|              pm25|time|
+-------+----------+------------------+------------------+----+
|  count|      4008|              4008|              4008|4008|
|   mean|      null|  62.6190119760479| 32.59855289421158|null|
| stddev|      null|32.331941444276644|19.541902861764598|null|
|    min|2015/01/02|               1.0|               0.0|hr_0|
|    max|2015/12/29|             204.0|             114.0|hr_9|
+-------+----------+------------------+------------------+----+

